In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [2]:
columns=['Ticker','Date','Close','Volume','EPS','B/S','B/P','E/P','S/P','Shares Float','Share Out','Dividend/Share','Shareholder yield','Cashflow','EBITDA/Share','Market,''Market Cap','Momentum 5 days','Momentum 20 days','Momentum 60 days','Momentum 120 days','RSI 14 days','Industry_1','Industry_2','Industry_3']

In [3]:
path=os.path.abspath("/Desktop/rank_stock.csv")

In [4]:
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Máy tính\Stock_ranking\rank_stock.csv",on_bad_lines = 'warn',parse_dates=True)
df.reset_index(inplace=True)
df=df[df['Industry_1'].notnull()]
df.drop(index=df.index[0], axis=0, inplace=True)
df=df.fillna(value=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_6088\554264271.py:1: DtypeWarning: Columns (13,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Máy tính\Stock_ranking\rank_stock.csv",on_bad_lines = 'warn',parse_dates=True)


In [5]:
df.set_index("index",inplace=True)

In [6]:
df['Cash flow']=pd.to_numeric(df['Cash flow'])

In [7]:
df.columns

Index(['Ticker', 'Date/Time', 'Close', 'Volume', 'EPS', 'B/S', 'B/P', 'E/P',
       'S/P', 'Shares Float', 'Shares Out', 'Dividend Per Share',
       'Shareholder Yield', 'Cash flow', 'EBITDA/Share', 'Market',
       'Market Cap', 'Momentum 5 days', 'Momentum 20 days', 'Momentum 60 days',
       'Momentum 120 days', 'RSI 14 days', 'Industry_1', 'Industry_2',
       'Industry_3'],
      dtype='object')

In [8]:
df['CF/P']=df['Close']/df['Cash flow']
df['EBITDA/EV']=df['EBITDA/Share']/df['Market Cap']

In [9]:
df['Trading Volume']= df['Close']*df['Volume']
df['Av_trading_60']=df['Trading Volume'].rolling(window=60,min_periods=20).mean()
df['Av_trading_20']=df['Trading Volume'].rolling(window=20,min_periods=20).mean()

In [10]:
cols_rank=['EPS','B/S','B/P','E/P','S/P','Shareholder Yield','Market Cap','Momentum 5 days','Momentum 20 days','Momentum 60 days','Momentum 120 days','CF/P','EBITDA/EV']

In [11]:
df['Market_cap']=(df['Shares Float']+df['Shares Out'])*df['Close']
df['Av_marketcap_60']=df['Market Cap'].rolling(window=60,min_periods=20).mean()
df['Av_marketcap_20']=df['Market Cap'].rolling(window=20,min_periods=20).mean()

In [12]:
df_rank=df.groupby(by=['Date/Time'])[cols_rank].rank()

In [13]:
df['valuation_score']=df_rank['B/P']+df_rank['S/P']+df['E/P']+df_rank['Shareholder Yield']+df_rank['CF/P']+df_rank['EBITDA/EV']

In [14]:
df_1=df[['Date/Time','Industry_1','Momentum 60 days']]
df_1

,Date/Time,Industry_1,Momentum 60 days
index,,,
251,9/6/2022 0:00,Hoa chat co ban - San pham nhua,0.00
252,9/7/2022 0:00,Hoa chat co ban - San pham nhua,0.00
253,9/8/2022 0:00,Hoa chat co ban - San pham nhua,0.00
254,9/9/2022 0:00,Hoa chat co ban - San pham nhua,0.00
255,9/12/2022 0:00,Hoa chat co ban - San pham nhua,0.00
...,...,...,...
1048570,8/29/2023 0:00,Hoa chat co ban - San pham nhua,-15.63
1048571,8/30/2023 0:00,Hoa chat co ban - San pham nhua,-18.78
1048572,8/31/2023 0:00,Hoa chat co ban - San pham nhua,-15.15


In [15]:
sector_perf=pd.DataFrame(df_1.groupby(by=['Industry_1','Date/Time']).mean()).reset_index().rename(columns={'Momentume_60':'Sector_lv2_perf_60'})

In [16]:
sector_perf=sector_perf.set_index(['Industry_1','Date/Time'])

In [17]:
sector_perf=sector_perf.rank()

In [18]:
sector_perf.reset_index(inplace=True)

In [19]:
df['ta_rating']=(df_rank['Momentum 20 days']+df_rank['Momentum 120 days'])/2

In [20]:
df['Rating']=2/3*(0.7*df['valuation_score']+0.3*df['ta_rating'])+1/3*sector_perf['Momentum 60 days']

In [24]:
from score_stock import penalty_funct

In [25]:
df['Modified_Rating'] = df.apply(lambda row: penalty_funct(row, 'Rating'), axis=1)

AttributeError: 'float' object has no attribute 'any'